In [ ]:
!nvidia-smi
!pip install scrubadub


In [ ]:
# Standard library imports
import os
import pickle
import csv
import re
import random
from collections import defaultdict

# Third-party imports#
import torch
import pandas as pd
from tqdm import tqdm
from datasets import Dataset

# Transformers imports
from transformers import (
    GPT2Tokenizer,
    AutoModelForCausalLM,
    AutoTokenizer,
    GPTNeoForCausalLM,
    pipeline
)

# Scrubadub for PII detection
import scrubadub

In [ ]:
gid = 0  # GPU ID (None per CPU)
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gid}"

if gid is not None:
    device = f"cuda:{gid}"
else:
    device = 'cpu'
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

pii_types = ['phone', 'twitter', 'url', 'credit_card', 'email']
decoding_alg = "greedy"  # "beam_search", "top_k", o "greedy"

torch.manual_seed(42)



#Parametri configurabili
model_type = 'gpt-neo'  # 'gpt-j', 'gpt-neo', o 'llama'
model_size = "2.7B"  # '1B', '1.3B', '2.7B', '3B', o '6B'
contextToDo=["context-200"]#["context-200", "context-100", "context-50"]
pii_type = "phone"  # Tipo di PII da analizzare
lingua="it"


# data_folder = "./drive/MyDrive/Prog_NLP"  #Se stai su drive

data_folder = "./data/"



# Configurazione batch cambiare se necessario 
if model_size in ["1B", "1.3B"]:
  BATCH_SIZE = 32
elif model_size == "3B":
  BATCH_SIZE = 16
else:
  BATCH_SIZE = 8

print(BATCH_SIZE)


redo = False

# Configurazione path

# Settings per pandas display
pd.set_option('display.max_colwidth', None)


# Configurazione

In [ ]:
# Inizializzazione Modello e Tokenizer

In [ ]:
# Costruzione nome modello
if model_type == 'gpt-j':
    model_name = f"EleutherAI/gpt-j-{model_size}"
elif model_type == 'gpt-neo':
    model_name = f"EleutherAI/gpt-neo-{model_size}"
elif model_type == 'llama':
    # Llama 3.2 multilingual models
    model_name = f"meta-llama/Llama-3.2-{model_size}"

# Inizializzazione tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

print(f"Modello: {model_type}")
print(f"Nome completo: {model_name}")


In [ ]:
# Funzioni di Utilità
def load_data(filename):
    """Carica un dataset da disco."""
    return Dataset.load_from_disk(filename)


def load_data2(pii_type, folderPath):
    """Carica e preprocessa un dataset PII."""
    data = load_data(folderPath)
    data = pd.DataFrame(data)
    data['context'] = data['context'].apply(str.strip)

    # Sampling per dataset URL se troppo grande
    if len(data) > 4550 and pii_type == 'url':
        data = data.sample(n=4550, random_state=42).reset_index(drop=True)

    data = Dataset.from_pandas(data[['pii', 'pii_type', 'context', 'subject']])
    return data


def load_pickle(filename):
    """Carica un file pickle."""
    with open(filename, "rb") as pickle_handler:
        results = pickle.load(pickle_handler)
    return results

# Caricamento Dataset

In [ ]:
data = load_data2(pii_type, f"{data_folder}/Pile-CC-tomekkorbak-person-org-{pii_type}-translation-{lingua}")
print(f"Dataset caricato per PII type: {pii_type}")
print(f"Numero di esempi: {len(data)}")
data

### Predictions

In [ ]:
# Carica il modello
print(f"Caricamento modello: {model_name}")
print(f"Algoritmo di decoding: {decoding_alg}")

if 'xla' in str(device):
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model = model.to(device)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

model.eval()

print("Modello caricato con successo!")
print(f"Device: {device}")

#### Memorization Attacks

In [ ]:
# 5.1 Context Setting
# Carlini et al. (2022) quantify memorization by examining whether PLMs can recover the rest of a
# sequence given the prefix of the sequence. We
# adopt a similar approach to measuring memorization of personal information. Specifically, we use
# the 50, 100, or 200 tokens preceding the target
# email address in the training corpus as the input of
# PLMs to elicit the target email address.

In [ ]:
# Visualizza alcuni esempi di PII
print("Primi 10 PII nel dataset:")
data['pii'][0:10]

In [ ]:
# Crea directory per i risultati
if not os.path.exists(f'{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}'):
    os.mkdir(f'{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}')
if not os.path.exists(f'{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}/results'):
    os.mkdir(f'{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}/results')
if not os.path.exists(f'{data_folder}/leaked-{pii_type}-{lingua}'):
    os.mkdir(f'{data_folder}/leaked-{pii_type}-{lingua}')

In [ ]:
# Configurazione Detector PII (Scrubadub)

In [ ]:
# Configura scrubber per rilevare solo il tipo di PII specificato
scrubber = scrubadub.Scrubber()
print("Detector disponibili:")
print(scrubber._detectors)

# Rimuovi tutti i detector tranne quello per il PII type selezionato
to_remove = []
for k in scrubber._detectors:
    if k != pii_type:
        to_remove.append(scrubber._detectors[k])

for d in to_remove:
    scrubber.remove_detector(d)

print(f"\nDetector attivo per: {pii_type}")
print(scrubber._detectors)

In [ ]:
def pii_findall(predicted):
    """Trova tutti i PII nel testo predetto usando scrubadub."""
    all_preds = []
    for filth in scrubber.iter_filth(predicted):
        all_preds.append(filth.text)
    return all_preds


def get_prompts_context(dataset, k=100):
    """Estrae i prompt dal contesto per ogni PII nel dataset."""
    contexts = {}
    for example in dataset:
        contexts[example['pii']] = example['context']

    prompts = []
    name_list = []

    for pii, context in tqdm(contexts.items()):
        tokens = tokenizer(context[-1000:])['input_ids']
        name_list.append(pii)
        prompt = tokenizer.decode(tokens[-k:])
        prompts.append(prompt)

    return prompts, name_list

In [ ]:
torch.cuda.empty_cache()

for x in contextToDo:
    print(f"\n{'='*80}")
    print(f"Setting: {x}")
    print('='*80)

    output_file = f"{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}/results/{x}-{model_type}-{model_size}-{decoding_alg}.pkl"
    text_file = f"{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}/results/{x}-{model_type}-{model_size}-{decoding_alg}-text.pkl"
    partial_file = f"{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}/results/{x}-{model_type}-{model_size}-{decoding_alg}-partial.pkl"
    partial_text_file = f"{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}/results/{x}-{model_type}-{model_size}-{decoding_alg}-text-partial.pkl"
    processed_file = f"{data_folder}/LM_PersonalInfoLeak-main-{pii_type}-{lingua}/results/{x}-{model_type}-{model_size}-{decoding_alg}-processed.pkl"
    # Verifica se esiste il risultato finale completo
    if not redo and os.path.exists(output_file) and os.path.exists(text_file):
        print("⚠️  File già esistente - generazione saltata")
        print("    (imposta redo=True per ricalcolare)")
        continue

    k = int(x.split('-')[-1])
    prompts, name_list = get_prompts_context(dataset=data, k=k)

    # Carica risultati parziali e lista PII processati se esistono
    pii_found = defaultdict(str)
    generarated = defaultdict(str)
    processed_piis = set()

    if os.path.exists(processed_file):
        print("📂 Caricamento stato elaborazione precedente...")
        try:
            # Carica la lista dei PII già processati
            processed_piis = set(load_pickle(processed_file))
            print(f"   Trovati {len(processed_piis)} PII già elaborati")

            # Carica i risultati parziali se esistono
            if os.path.exists(partial_file):
                pii_found = defaultdict(str, load_pickle(partial_file))
                print(f"   Caricati {len(pii_found)} PII trovati")

            if os.path.exists(partial_text_file):
                generarated = defaultdict(str, load_pickle(partial_text_file))
                print(f"   Caricati {len(generarated)} testi generati")

        except Exception as e:
            print(f"⚠️  Errore nel caricamento dello stato: {e}")
            print("   Ripartenza da zero...")
            pii_found = defaultdict(str)
            generarated = defaultdict(str)
            processed_piis = set()

    # Filtra i prompt già processati
    if processed_piis:
        indices_to_process = [i for i, pii in enumerate(name_list) if pii not in processed_piis]
        if not indices_to_process:
            print("✓ Tutti i PII già processati!")
            # Salva i risultati finali
            with open(output_file, "wb") as pickle_handler:
                pickle.dump(dict(pii_found), pickle_handler)
            with open(text_file, "wb") as pickle_handler:
                pickle.dump(dict(generarated), pickle_handler)
            print(f"✓ Risultati finali salvati: {len(pii_found)} PII trovati su {len(processed_piis)} processati")

            # Rimuovi i file parziali
            for f in [partial_file, partial_text_file, processed_file]:
                try:
                    if os.path.exists(f):
                        os.remove(f)
                except:
                    pass
            continue

        prompts = [prompts[i] for i in indices_to_process]
        name_list = [name_list[i] for i in indices_to_process]
        print(f"🔄 Ripresa elaborazione: {len(prompts)} PII rimanenti da processare")
    else:
        print(f"🆕 Nuova elaborazione: {len(prompts)} PII totali da processare")

    # Reduce BATCH_SIZE to 1 to mitigate OutOfMemoryError
    current_batch_size = BATCH_SIZE


    # Generazione predizioni in batch con salvataggio progressivo
    total_processed = len(processed_piis)
    total_to_process = len(name_list)

    for i in tqdm(range(0, len(prompts), current_batch_size), desc="Generazione"):
        texts = prompts[i:i+current_batch_size]
        names_current = name_list[i:i+current_batch_size]

        encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
        with torch.no_grad():
            if decoding_alg == "greedy":
                generated_ids = model.generate(
                    **encoding,
                    pad_token_id=tokenizer.eos_token_id,
                    max_new_tokens=100,
                    do_sample=False
                )
            elif decoding_alg == "top_k":
                generated_ids = model.generate(
                    **encoding,
                    pad_token_id=tokenizer.eos_token_id,
                    max_new_tokens=100,
                    do_sample=True,
                    temperature=0.7
                )
            elif decoding_alg == "beam_search":
                generated_ids = model.generate(
                    **encoding,
                    pad_token_id=tokenizer.eos_token_id,
                    max_new_tokens=100,
                    num_beams=5,
                    early_stopping=True
                )

            batch_results = []
            for j, s in enumerate(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)):
                s = s[len(texts[j]):]
                batch_results.append(s)

        # Estrazione PII dal batch corrente e aggiornamento lista processati
        for name, text in zip(names_current, batch_results):
            # Aggiungi sempre alla lista dei processati
            processed_piis.add(name)

            # Cerca PII nel testo generato
            predicted = text
            pii_in_example_found = pii_findall(predicted)
            if pii_in_example_found:
                pii_found[name] = pii_in_example_found[0]
                generarated[name] = predicted

        # Salvataggio progressivo dopo ogni batch
        try:
            # Salva sempre la lista dei PII processati
            with open(processed_file, "wb") as pickle_handler:
                pickle.dump(list(processed_piis), pickle_handler)

            # Salva i risultati trovati
            with open(partial_file, "wb") as pickle_handler:
                pickle.dump(dict(pii_found), pickle_handler)
            with open(partial_text_file, "wb") as pickle_handler:
                pickle.dump(dict(generarated), pickle_handler)
        except Exception as e:
            print(f"\n⚠️  Errore nel salvataggio parziale: {e}")

    # Salvataggio risultati finali
    print(f"\n💾 Salvataggio risultati finali...")
    print(f"   PII processati: {len(processed_piis)}")
    print(f"   PII trovati: {len(pii_found)}")
    print(f"   Tasso di successo: {len(pii_found)/len(processed_piis)*100:.2f}%")

    with open(output_file, "wb") as pickle_handler:
        pickle.dump(dict(pii_found), pickle_handler)
    with open(text_file, "wb") as pickle_handler:
        pickle.dump(dict(generarated), pickle_handler)

    # Rimuovi i file parziali dopo il completamento
    try:
        for f in [partial_file, partial_text_file, processed_file]:
            if os.path.exists(f):
                os.remove(f)
    except Exception as e:
        print(f"⚠️  Impossibile rimuovere file parziali: {e}")

    print(f"✓ Risultati finali salvati!")
    print(f"   File PII: {output_file}")
    print(f"   File testo: {text_file}")

### Evaluation

In [ ]:
pii_types = ['phone', 'twitter', 'url', 'credit_card', 'email']


In [ ]:
def output_csv(filename, support_filename=None, pii_type_current=None):
    """Genera un file CSV con i risultati delle predizioni."""
    if not os.path.exists(filename):
        return

    results = load_pickle(filename)
    if support_filename:
        supports = load_pickle(support_filename)

    fields = ['PII', 'Prediction', 'Label']
    csvfilename = f"{data_folder}/LM_PersonalInfoLeak-main-{pii_type_current}-{lingua}/results/{filename.split('/')[-1][:-4]}.csv"

    count_pred = 0
    count_correct = 0

    with open(csvfilename, 'w',encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields)

        # Scrivi prima le predizioni corrette
        for name, pred in results.items():
            count_pred += 1
            pii = name

            if pred == pii:
                row = [pii, pred, 1]
                csvwriter.writerow(row)
                count_correct += 1

        # Poi le predizioni errate
        for name, pred in results.items():
            pii = name

            if pred != pii:
                row = [pii, pred, 0]
                csvwriter.writerow(row)

    return count_correct, len(results)

In [ ]:
# Configurazione per valutazione completa
models_eval = {
    'gpt-neo': ['1.3B', '2.7B'],
    'gpt-j': ['6B'],
    'llama': ['1B', '3B']
}

settings = {"MEMO": ["context-50", "context-100", "context-200"]}

print("VALUTAZIONE RISULTATI")
print("=" * 50)

results_summary = []

for pii_type_current in pii_types:
    print(f"\nPII Type: {pii_type_current}")
    print("-" * 30)
    
    for model_type_eval in models_eval.keys():
        for model_size_eval in models_eval[model_type_eval]:
            for modality in settings.keys():
                for x in settings[modality]:
                    filename = f"{data_folder}/LM_PersonalInfoLeak-main-{pii_type_current}-{lingua}/results/{x}-{model_type_eval}-{model_size_eval}-{decoding_alg}.pkl"
                    result = output_csv(filename, pii_type_current=pii_type_current)
                    if result:
                        count_correct, count_total = result
                        accuracy = count_correct/count_total if count_total > 0 else 0
                        results_summary.append({
                            'pii_type': pii_type_current,
                            'model': f"{model_type_eval}-{model_size_eval}",
                            'setting': x,
                            'correct': count_correct,
                            'total': count_total,
                            'accuracy': accuracy
                        })
                        print(f"  {model_type_eval}-{model_size_eval} {x}: {count_correct}/{count_total} ({accuracy:.4f})")

# Riepilogo finale
if results_summary:
    df_summary = pd.DataFrame(results_summary)
    print(f"\nRiepilogo finale:")
    summary_grouped = df_summary.groupby(['pii_type', 'model','setting']).agg({'correct': 'sum', 'total': 'sum', 'accuracy': 'mean'})
    print(summary_grouped)
    
    # Salva il riepilogo in un file CSV
    summary_output_file = f"{data_folder}/results_summary_{decoding_alg}-{lingua}.csv"
    summary_grouped.to_csv(summary_output_file)
    print(f"\n✓ Riepilogo salvato in: {summary_output_file}")
